# <center> DENİZ BALCI SİMÜLASYON 2.ÖDEVİ</center>

# 0)  Paket Kurulumu ve Ön açıklama

Bütün hücreleri ayrı ayrı veya bütün halde çalıştırabilirsiniz.İlk olarak paketleri yükleyelim.

In [1]:
!pip install scipy
!pip install numpy
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


## SORU 1


Belirli bir hızla hareket eden bir aracın fren mesafesi metre cinsinden X rasgele değişkeni ile gösterilsin. X’in dağılım fonksiyonunun


 \begin{equation}
 F(x) =
   \left\{\begin{array}{lr}
       0, & x \le 3 \\
      \frac{x^2-6x+9}{4}  & 3 < x < 5 \\
      1 & x \ge 5
    \end{array}\right.
 \end{equation}
 
 
<ol style="list-style-type: lower-alpha;">
  <li>Aracın en fazla 2m'de durabilme olasılığı  <p style="color:red"> !   SORUDAKİ HATA ŞU SINIRLAR 3 ile 5 arasında ancak 2 de durabilme olasılığında soruyorsunuz.</p> </li>
  <li>Aracın %90 olasılıkla durabilmesi için fren mesafesinin alt sınırını simülasyon teknikleri yardımıyla bulunuz.</li>

</ol> 

 İlk olarak elimizdeki dağılımın özelliklerini ve ters dönüşüme sahip olup olmadığını inceleyelim.
 
 \begin{equation}
 f(x) =
   \left\{\begin{array}{lr}
      \frac{2x-6}{4}  & 3 < x < 5 \\
    \end{array}\right.
 \end{equation}
  \begin{equation}
 F^{-1}(x) =
   \left\{\begin{array}{lr}
      \sqrt{4y}+3  & 0 < x <1 \\
    \end{array}\right.
 \end{equation}
 
 
 $$ E(x)=4.33 \quad  Var(x)=39.17$$
 
 Görüldüğü gibi ters dönüşüm fonksiyonunu kullanabiliriz.

In [32]:
import numpy as np
import scipy.stats
import math

def ters_fonksiyon(p:float)->float:
    assert 0 <= p <= 1 
    sonuc=np.sqrt(4*p)+3
    return sonuc


def veri_üretici(n: int)->list:
    assert n > 0
    uniform_dataset = np.random.uniform(0, 1, n)
    dataset = [ters_fonksiyon(i) for i in uniform_dataset]
    sorted_dataset = sorted(dataset)  # Sort the dataset
    return sorted_dataset
    

def oran(lookingvalue:float,veriseti)->float:
    n =len(veriseti)
    ratio = sum(value <lookingvalue for value in veriseti)/n
    return ratio
    


    
n = int(100)
p = 0.90
num_samples = int(n * p)
veriseti=veri_üretici(n)    

print("A sorusunun yanıtı 2 metreden az olma olasılığı \n")
print(oran(2,veriseti))
print("\n")

print("B Aracın %90 olasılıkla durabilmesi için fren mesafesinin alt sınırı \n")
print(veriseti[num_samples])
print("\n")




A sorusunun yanıtı 2 metreden az olma olasılığı 

0.0


B Aracın %90 olasılıkla durabilmesi için fren mesafesinin alt sınırı 

4.888214260815366




## SORU 2


Kabul-Red yöntemini açıklayınız ve bir tane hem ters dönüşüm tekniği,hem kabul red yöntemi ile veri üretebileceğiniz dağılım bulunuz.
n=10,50,100,1000 örneklem büyükleri için örneklem ortalaması ve anakütle ortalaması değerleri ile karşılaştırınız.



### Kabul red yöntemi
Kabul red yöntemi ters dönüşüm uygulayamadığımız durumlarda kullanılan bir methoddur. Bu metodda, bir olasılık yoğunluk fonksiyonunu simule etmek için ek olarak bilinen başka bir fonksiyon üzerinden  oranlayarak kabul bölgesi oluşturduğumuz bir yapıdır.


\begin{equation}
u \le \frac{f(x)}{c*g(x)}
\end{equation}

<center><img src="img/ikinciodev/kabuleksik.png" alt="MarineGEO circle logo" style="height: 500px; width:1000px;"/>

YUKARIDAKİ GRAFİKTE MOR ALANLAR RED EDİLEN ALANLARDIR.


YEŞİL ALAN HİPOTEZİN  KABUL EDİLEN ALANLARDIR.



</center>

### ÖRNEĞİMİZE DÖNELİM


In [25]:
import numpy as np
import scipy.stats
import math
import matplotlib.pyplot as plt

def inverse_F(x, lambdaa):
    return -lambdaa * math.log(1 - x)


def ters_veri_üretici(n: int, lambdaa)->list:
    assert n > 0
    uniform_dataset = np.random.uniform(0, 1, n)
    dataset = [inverse_F(i, lambdaa) for i in uniform_dataset]
    sorted_dataset = sorted(dataset)  # Sort the dataset
    return sorted_dataset


def f(x, lambdaa):
    return (1/ lambdaa) * np.exp(-x/ lambdaa)

c = 1

def g(x):
    return 0.5 * np.exp(-x * 0.5)

def AR(N, lambdaa):
    x = np.array([])
    Y = np.random.exponential(lambdaa, 1)
    U = np.random.uniform(0, 1, 1)
    #print(f(Y,lambdaa) / (c * f(Y,lambdaa)))
    for i in range(N):
        while True:
            Y = np.random.exponential(lambdaa, 1)
            U = np.random.uniform(0, 1, 1)
            if U <= f(Y,2) / (c * g(Y)):
                x = np.append(x, Y)
                break
    return x  




In [31]:
lambdaa=4
n1=10
n2=50
n3=100
n4=1000
import pandas as pd



table = [[np.mean(AR(n1, lambdaa)), np.mean(AR(n1, lambdaa)), np.mean(AR(n3, lambdaa)), np.mean(AR(n4, lambdaa))],
         [np.mean(ters_veri_üretici(n1, lambdaa)), np.mean(ters_veri_üretici(n2, lambdaa)),
          np.mean(ters_veri_üretici(n3, lambdaa))
          , np.mean(ters_veri_üretici(n4, lambdaa))]]
df = pd.DataFrame(table, columns = [10,50,100,1000], index=['kabul-red', 'ters dönüşüm'])

print(df)


                  10        50        100       1000
kabul-red     3.567954  1.313683  4.077318  4.087985
ters dönüşüm  4.947460  3.666313  3.370973  4.246830


## SORU 3
Poisson $(\theta)$) dağılımından veri üretecek
<ol style="list-style-type: lower-alpha;">
  <li>$P(X \leq x )$</li>
 <li>$P(X = x )$</li>
<li>$P(X \ge x )$</li>
</ol> 
olasılıklarını bulan  bir program yazınız.Burada $\theta$ ve x değeri girdi olarak kullanıcıdan istenilecek.

## 1.YOL 

In [2]:
import math

class Poisson:
  def __init__(self,x,theta):
     self.theta=theta
     self.x=x
     
  def poisson(self):
      result= (math.exp(-self.theta)*(self.theta **self.x))/math.factorial(self.x)
      return result
  
  def littleeq(self):
     result = 0
     for i in range(0,self.x):
        result=result+self.poisson()
     return result
  
  def biggerthan(self):
     return 1-self.littleeq()
  

x=5
theta=2
p1 = Poisson(x, theta)
cdf = p1.littleeq()
print("P(X <= 5) =", cdf)
print("P(X = 5)  =", p1.poisson())
print("P(X > 5)  =", p1.biggerthan())

P(X <= 5) = 0.1804470443154836
P(X = 5)  = 0.03608940886309672
P(X > 5)  = 0.8195529556845164


## 2.YOL 

Notlardaki yol

In [47]:
import math
import random

def poisson(lambd):
    a = math.exp(-lambd)
    b = 1
    i = 0
    
    while True:
        ri_plus_1 = random.uniform(0, 1)
        b *= ri_plus_1
        
        if b < a:
            x = i
            break
        
        i += 1
    
    return x

# Örnek kullanım:
lambd = 2  # Poisson dağılımın lambda değeri
print(poisson(lambd))


3
